## Импорт и определение библиотек

In [1]:
# отключение параллелизма
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
# ставим разные пакеты оттуда
!pip install langchain langchain-openai langchain-community openai tiktoken langchain-huggingface -q
!pip install beautifulsoup4 pypdf sentence-transformers faiss-cpu unstructured pypandoc -q
#huggingface_hub  langchain_experimental langchainhub
#!pip install unstructured "unstructured[pdf]" -q
!pip install sentence-transformers numpy transformers requests -q

In [3]:
# импортируем библиотеки
from langchain.document_loaders import WebBaseLoader, PyPDFLoader, UnstructuredEPubLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEndpoint
from langchain.agents import Tool, initialize_agent
from langchain.prompts import PromptTemplate
import tqdm as notebook_tqdm
from huggingface_hub import HfApi

USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/elijah/Documents/LLM/cocktail_qr/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Готовим коктейль с помощью RAG
## Чутка `embeddings` с `HuggingFace` для оснастки

In [99]:
# Чтение токена из файла
def read_token(file_path): 
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith("HUGGINGFACEHUB_API_TOKEN"):
                return line.split('=')[1].strip()
    raise ValueError("Token not found in the specified file.")

# Загрузка токена
hf_token = read_token('../utils.key')

# Формирование эмбеддингов
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",  
    #"sentence-transformers/paraphrase-multilingual-mpnet-base-v2", 
    #"cointegrated/LaBSE-en-ru",
    model_kwargs={
        "device": "cpu",
        "token": hf_token
                  } 
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-mpnet-base-v2


## Можно пропустить (если используете наполненную чашу векторного хранилища)
### Немного `Document Loader`

In [100]:
# Загрузка EPUB файла от бармена-литературоведа
loader = UnstructuredEPubLoader('../docs/Federle.epub', show_progress=True)
doc_epub = loader.load()

# смотрим, скока получилось загрузить
print('pages count:', len(doc_epub))
print('max chars on page:', max([len(page.page_content) for page in doc_epub]))

pages count: 1
max chars on page: 107261


In [ ]:
# Загружаем книгу от крутого бармена
#loader = WebBaseLoader(url)
"""loader = PyPDFLoader('../docs/Bortnik_1000.pdf')
data = loader.load()

print('pages count:', len(data))
print('max chars on page:', max([len(page.page_content) for page in data]))"""

# на тот случай, если решим загружать подобную литературу пачкой pdf
"""
#!pip install unstructured "unstructured[pdf]" -q
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../docs", glob="**/*.pdf", show_progress=True)
data = loader.load()"""

### Исполним `Text splitters` и разобьём на чанки

**Точная токенизация под `DeepSeek`**

- Кастомный класс `TransformersTokenSplitter`:
  - принимает токенизатор `Hugging Face`.
  - токенизирует текст с помощью `tokenizer.encode()`.
  - разбивает токены на чанки с учетом `chunk_size` и `chunk_overlap`.
  - декодирует чанки обратно в текст через `tokenizer.decode()`.

- Особенности:
  - точное соответствие токенизации модели `DeepSeek`.
  - сохранение перекрытия между чанками.
  - поддержка любых токенизаторов из `Transformers`.

- Почему это лучше стандартного `TokenTextSplitter`:
  - точность: используется токенизатор, идентичный модели.
  - гибкость: работает с любыми моделями `Hugging Face`.
  - контроль: позволяет задавать параметры в токенах, а не символах.
  
Сравнение подходов:
| Параметр | Решение 1 (Символы) | Решение 2 (Токены)|
|---|---|---|
| Точность | Средняя	| Высокая |
|Производительность	 | Быстрее	| Медленнее (токенизация) |
|Сложность	| Просто	| Требует токенизатор |
|Рекомендация	| Для общего случая	| Для точного контекста |

In [6]:
from langchain.text_splitter import TextSplitter
from transformers import AutoTokenizer

class TransformersTokenSplitter(TextSplitter):
    """Кастомный сплиттер для токенизаторов Hugging Face."""
    
    def __init__(self, tokenizer, chunk_size=256, chunk_overlap=64):
        super().__init__(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        self.tokenizer = tokenizer

    def split_text(self, text: str) -> list[str]:
        # Разбиваем текст на части, если он слишком большой
        max_length = self.tokenizer.model_max_length
        if len(text) > max_length:
            # Разбиваем текст на части по max_length
            parts = [text[i:i + max_length] for i in range(0, len(text), max_length)]
        else:
            parts = [text]
        
        chunks = []
        for part in parts:
            # Токенизируем часть текста
            tokens = self.tokenizer.encode(part, add_special_tokens=False)
            
            # Разбиваем токены на чанки
            current_chunk = []
            current_length = 0
            
            for token in tokens:
                current_chunk.append(token)
                current_length += 1
                
                if current_length >= self._chunk_size:
                    # Декодируем чанк обратно в текст
                    chunk_text = self.tokenizer.decode(current_chunk)
                    chunks.append(chunk_text)
                    
                    # Сохраняем перекрытие
                    current_chunk = current_chunk[-self._chunk_overlap :]
                    current_length = len(current_chunk)
            
            # Добавляем последний чанк
            if current_chunk:
                chunk_text = self.tokenizer.decode(current_chunk)
                chunks.append(chunk_text)
                
        return chunks

# -----------------------------------------------------------
# Пример использования
# -----------------------------------------------------------

# Загрузка токенизатора DeepSeek
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-llm-7b-chat")

# Инициализация сплиттера
splitter = TransformersTokenSplitter(
    tokenizer=tokenizer,
    chunk_size=256,      # Размер чанка в токенах
    chunk_overlap=64     # Перекрытие между чанками
)

In [8]:
# Разделяем книгу на части

split_documents = []
for page in doc_epub:
    # Разбиваем текст страницы на чанки
    chunks = splitter.split_text(page.page_content)
    for chunk in chunks:
        # Сохраняем метаданные страницы
        split_documents += splitter.create_documents([page.page_content], metadatas=[page.metadata])

In [9]:
chunks = splitter.split_documents(doc_epub)
print('Количество чанков:', len(chunks))
print('Максимальное количество символов в чанке:', max([len(chunk.page_content) for chunk in chunks]))

Количество чанков: 260
Максимальное количество символов в чанке: 707


### Наполним чашу `Vector Store`

In [12]:
# Если нужно сохранить новый документ в базу
try:
    db_embed = FAISS.from_documents(
            split_documents, 
            hf_embeddings_model
        )
except Exception as e:
    raise RuntimeError(f"Ошибка при создании базы данных FAISS: {e}")


# Сохраняем векторную базу локально
db_embed.save_local("../data/faiss_db_epub")

INFO:faiss.loader:Loading faiss with AVX512 support.
INFO:faiss.loader:Successfully loaded faiss with AVX512 support.


## Нужно запустить (если используете готовое векторное хранилище)
### Испьём чашу `Vector Store`

In [101]:
# Если нужно изъять из уже сохранённого документа из базы
# Загрузка векторной базы данных из локального файла
db_embed = FAISS.load_local(
        "../data/faiss_db_epub", 
        hf_embeddings_model,
        allow_dangerous_deserialization = True
    )

In [102]:
query = "ржаной виски, грейпфрутовый сок"
results = db_embed.similarity_search(query, k=5)
for result in results:
    print(result.page_content)

НА ВОСЕМЬ ПОРЦИЙ:

350 мл ржаного виски;

120 мл ананасового сока;

60 мл лимонного сока;

1 л имбирного пива (истинно американский напиток, который непросто отыскать на наших просторах).

Виски и соки заливаем в чашу для пунша, где уже лежит глыба льда. Вливаем имбирное пиво, трубим общий сбор – пора развеять экзистенциальную тоску!

Приключения Шербета Холмса

ПРИКЛЮЧЕНИЯ ШЕРЛОКА ХОЛМСА (1891–1892)

СЭР АРТУР КОНАН ДОЙЛЬ
30 мл водки;

200 г нарезанного кубиками арбуза;

10 мл лимонного сока;

½ чайной ложки (2,5 г) сахара;

15 мл дынного ликера.

Водку, арбуз, лимонный сок, сахар и пригоршню льда помещаем в блендер и доводим до однородности. Переливаем в коктейльный бокал, аккуратно добавляем ликер. Глоток чудо-смузи подарит вам такую отвагу, что вы не побоитесь бросить вызов даже самому Главноуправителю.

Коктейль о двух городах

ПОВЕСТЬ О ДВУХ ГОРОДАХ (1859)

ЧАРЛЬЗ ДИККЕНС
60 мл клюквенного сока;

60 мл апельсинового сока;

50 мл светлого рома;

30 мл кокосового рома;

1 чайная ло

# Месье системный промт

In [103]:
# Системный промпт
system_prompt = """Ты — помощник для поиска рецептов коктейлей. Используй только данные из контекста ниже:
        {context}

        Пример:
        Пользователь: Какие коктейли можно сделать из водки и апельсинового сока?
        Агент: Использую инструмент "Cocktail Recipe Finder" для поиска рецептов. Вот что найдено: [рецепт].

        **Строгий формат ответа**:
        1. Название коктейля (только одно уникальное название из источника)
        2. Состав:
        - Ингредиент 1 (объем)
        - Ингредиент 2 (объем)
        3. Способ приготовления:
        - Шаг 1
        - Шаг 2

        **Запрещено**:
        - Добавлять описание вкуса, аромата или эмоций
        - Использовать сложные термины (например, \"вкусовые соединения\")
        - Создавать варианты в скобках

        Пример корректного ответа по введённым ингредиентам (ржаной виски, грейпфрутовый сок):
        Коктейль "Рожь и предубеждение"
        Состав:
        - Ржаной виски (50 мл)
        - Грейпфрутовый сок (90 мл)
        Способ приготовления:
        - Ингредиенты выливаем в стакан рокс поверх кубиков льда"""

# Промпт для LangChain
qa_prompt = PromptTemplate(
    template=system_prompt + "\nВопрос: {question}\nОтвет:",
    input_variables=["context", "question"]
)

## Полирнём `Retriver` и инициализирует `llm` для вкуса

In [16]:
!pip install langchain ollama -q

In [104]:
# Инициализация модели DeepSeek
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Инициализация модели через Ollama
llm = Ollama(
    model="deepseek-llm:7b-chat",
    temperature=0.3,        # Увеличиваем для большей креативности
    num_predict=512,        # Уменьшаем длину ответа
    repeat_penalty=1.2,     # Усиливаем штраф за повторы
    #top_p=0.9,              # Для фокусировки
    #top_k=40,               # Ограничиваем выбор токенов
    stop=["\\n\\n"]         # Остановка при двойных переносах
)

In [105]:
# Модифицируем параметры ретривера
retriever = db_embed.as_retriever(
    search_type="mmr",              # Maximal Marginal Relevance
    search_kwargs={
        "k": 3,                     # Уменьшаем количество возвращаемых документов
        "fetch_k": 10,              # Уменьшаем общий пул для поиска
        "lambda_mult": 0.6          # Баланс между релевантностью и разнообразием
    }
)


# Инициализация цепочки RetrievalQA
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,  # Ваш ретривер (например, FAISS)
    chain_type_kwargs={"prompt": qa_prompt},
    return_source_documents=True
)

# Миксуем рецепт коктейля `agent` 00х

In [106]:
# Создание инструмента
agent_tools = [
    Tool(
        name="Cocktail Recipe Finder",
        func=retrieval_qa.run,
        description="Используй этот инструмент, чтобы найти рецепты коктейлей по заданным ингредиентам. Вводи список ингредиентов, и инструмент вернёт подходящие рецепты. Пример: ['сок', 'виски']."
    )
]

# создание агента
agent = initialize_agent(
    tools=agent_tools, 
    llm=llm, 
    agent="conversational-react-description",       #"zero-shot-react-description", "plan-and-execute"
    verbose=True,
    system_prompt=system_prompt                     # не забыть указать системный промт для вкуса
)

# Задаём общие параметры, после чего коктейль готов
## Вначале коктейль

In [107]:
# Объявление функции на ввод данных от пользователя
def get_user_input():
    """
    Запрашивает у пользователя ингредиенты для поиска рецептов.
    """
    user_input = input("Введите ингредиенты, разделенные запятыми: ")
    ingredients = [ingredient.strip() for ingredient in user_input.split(",")]
    return ingredients


# Обработка ответа
def format_response(response):
    if "не знаю" in response.lower() or "не найдено" in response.lower():
        return "К сожалению, я не нашёл коктейлей с этими ингредиентами. Попробуйте другие ингредиенты."
    else:
        return response
    

# Функция выбора стиля ответа
def choose_style(styles):
    chosen_style = input("Выберите номер стиля (по умолчанию 4): ") or "4"
    return styles.get(chosen_style, "по-умолчанию")


In [121]:
def main():
    # Стилевые инструкции с акцентом на общие характеристики
    style_instructions = {
        "1": {
            "name": "Космический ужас Лавкрафта",
            "description": (
                "Используй архаичную лексику, метафоры древних сил и непостижимых существ. "
                "Допустимые элементы: нектар тьмы, дрожащие ингредиенты, запретные ритуалы. "
                "Избегай бытовых описаний."
            ),
            "params": {"temperature": 0.8, "top_p": 0.85}
        },
        "2": {
            "name": "Гопническо-быдляцкий жаргон",
            "description": (
                "Используй грубую лексику, просторечия и абсурдные сравнения. "
                "Примеры: 'замути', 'вмазать', 'как пацанчик'. "
                "Сокращай предложения."
            ),
            "params": {"temperature": 0.9, "top_p": 0.95}
        },
        "3": {
            "name": "Экспериментальный стиль Берроуза",
            "description": (
                "Разрывай текст на фрагменты, используй нелинейность и аллюзии. "
                "Пример: 'Шейкер*вибрация*льдинки-глаза*взрыв*смешение'."
            ),
            "params": {"temperature": 1.0, "top_p": 0.99}
        },
        "4": {
            "name": "Стандартный",
            "description": "Без изменений",
            "params": {"temperature": 0.0, "top_p": 0.95}
        }
    }

    while True:
        ingredients = get_user_input()
        if not ingredients:
            print("Вы не ввели ингредиенты. Попробуйте снова.")
            continue

        # Базовый запрос
        ingredients_str = ", ".join(ingredients)
        user_prompt = f"""
        Найди рецепты коктейлей по ингредиентам: {ingredients_str}
        Учти соответствие с системным промтом и стилистикой ответа.
        """

        response = retrieval_qa.invoke({"query": user_prompt})
        original_response = response["result"]

        # Выбор стиля
        chosen_style = input("Выберите стиль (1-4): ") or "4"
        style_config = style_instructions.get(chosen_style, style_instructions["4"])
        
        # Промпт для стилизации
        style_prompt = f"""Перепиши этот текст в заданном стиле: {original_response}\nСтиль: {style_instructions}.\nРезультат:\n''
        """

        # Генерация с настройками стиля
        styled_response = llm.invoke(
            style_prompt,
            temperature=style_config["params"]["temperature"],
            top_p=style_config["params"]["top_p"]
        )

        # Постобработка
        styled_response = styled_response.strip()

        print(styled_response)
        break

    return styled_response

if __name__ == "__main__":
    # Сохраняем результат в переменную
    recipe = main()

1. Бурбон в летнюю ночь (ок. 1600)
    * Шеекер
      - 50 мл дешевого виски; сплэш ананасового сока (это означает плеснуть 5–10 мл); шлепок кокосовых сливок.
      - Ингредиенты наливаем поверх кубиков льда в стакан рокс (впрочем, подойдет и туристическая кружка). Энергично перемешиваем, хватаем с полки "Таитянский для начинающих" и мчим в аэропорт!
2. Прощай, "Амаретто"! (1929)
    * Шеекер
      - 60 мл водки; 15 мл сока лайма; ½ чайной ложки (2,5 г) вустерского соуса; ½ чайной ложки (2,5 г) васаби; 12–15 капель острого соуса (3 дэша на языке барменов); соль и перец по вкусу.
      - Все ингредиенты соединяем в шейкер с льдом. От души взбалтываем и процеживаем через стрейнер в стакан коллинз на свежеприготовленный лед. Поборники традиций украшают напиток стебельком сельдерея, но если вы любите "погорячее", то просто удвойте порцию васаби.
3. Говардс-бленд (1910)
    * Форстэсер
      - Эдвин Морган Фостер


## Теперь озвучка

In [113]:
# импортируем библиотеки
import requests
import uuid

# функция для получения токена
def get_token(auth_token, scope='SALUTE_SPEECH_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «SALUTE_SPEECH_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {
        'scope': scope
    }

    try:
        # Делаем POST запрос с отключенной SSL верификацией
        # (можно скачать сертификаты Минцифры, тогда отключать проверку не надо)
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return None

In [114]:
# Считываем ключ авторизации из файла
with open('../utils.key', 'r') as file:
    for line in file:
        if line.startswith('SBER_API_KEY'):
            SBER_API_KEY = line.split('=', 1)[1].strip(' \n\r')
            if not SBER_API_KEY.endswith('=='):  # Если нет ==, добавляем их
                SBER_API_KEY += '=='
            break

print(f"Прочитанное значение: '{SBER_API_KEY}'")
#'MTVjODUyZDMtZTU2Mi00ZWY5LWI3YzctZGY4YjUxZGMzMjhjOjllZmQ3NjZmLTVhYWQtNGYxOC1iN2I4LTQwOGMyY2IwZWI4Yg=='

response = get_token(SBER_API_KEY) # получаем токен
if response != None:
    salute_token = response.json()['access_token']

Прочитанное значение: 'MTVjODUyZDMtZTU2Mi00ZWY5LWI3YzctZGY4YjUxZGMzMjhjOjllZmQ3NjZmLTVhYWQtNGYxOC1iN2I4LTQwOGMyY2IwZWI4Yg=='


/home/elijah/Documents/LLM/cocktail_qr/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [115]:
# функция для синтеза речи
def synthesize_speech(text, token, format='wav16', voice='May_24000'):
    url = "https://smartspeech.sber.ru/rest/v1/text:synthesize" # URL для синтеза речи
    headers = {
        "Authorization": f"Bearer {token}", # токен авторизации
        "Content-Type": "application/text" # тип содержимого
    }
    params = {
        "format": format, # формат аудио
        "voice": voice # голос
    }
    response = requests.post(
            url, 
            headers=headers, 
            params=params, 
            data=text.encode(), # текст для синтеза
            verify=False # отключение проверки SSL
        )

    # проверка на успешность синтеза
    if response.status_code == 200:
        # Сохранение синтезированного аудио в файл
        with open('output.wav', 'wb') as f:
            f.write(response.content)
        print("Аудио успешно синтезировано и сохранено как 'output.wav'")

In [116]:
# вызов функции для синтеза речи
synthesize_speech(recipe, salute_token)

/home/elijah/Documents/LLM/cocktail_qr/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Аудио успешно синтезировано и сохранено как 'output.wav'
